# Workshop "Data Ingestion with dlt": Homework

---

## **Dataset & API**

We’ll use **NYC Taxi data** via the same custom API from the workshop:

🔹 **Base API URL:**  
```
https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api
```
🔹 **Data format:** Paginated JSON (1,000 records per page).  
🔹 **API Pagination:** Stop when an empty page is returned.

## **Question 1: dlt Version**

1. **Install dlt**:


In [1]:
pip install "dlt[duckdb]"

  Using cached dlt-1.6.1-py3-none-any.whl (884 kB)
  Using cached tomlkit-0.13.2-py3-none-any.whl (37 kB)
  Using cached makefun-1.15.6-py2.py3-none-any.whl (22 kB)
  Using cached pluggy-1.5.0-py3-none-any.whl (20 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl (184 kB)
  Using cached click-8.1.8-py3-none-any.whl (98 kB)
  Using cached hexbytes-1.3.0-py3-none-any.whl (4.9 kB)
  Using cached pendulum-3.0.0-cp310-cp310-macosx_11_0_arm64.whl (352 kB)
  Using cached setuptools-75.8.0-py3-none-any.whl (1.2 MB)
  Using cached GitPython-3.1.44-py3-none-any.whl (207 kB)
  Using cached humanize-4.12.0-py3-none-any.whl (127 kB)
  Using cached jsonpath_ng-1.7.0-py3-none-any.whl (30 kB)
  Using cached pathvalidate-3.2.3-py3-none-any.whl (24 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
  Using cached giturlparse-0.12.0-py2.py3-none-any.whl (15 kB)
  Using cached simplejson-3.20.1-cp310-cp310-macosx_11_0_arm64.whl (75 kB)
  Using cached PyYAML-6.0.2-cp310-cp310-macosx_11_0_arm64.wh

**2. Check the version:**

In [2]:
!dlt --version

dlt 1.6.1


## **Question 2: Define & Run the Pipeline (NYC Taxi API)**

Use dlt to extract all pages of data from the API.

Steps:

1️⃣ Use the `@dlt.resource` decorator to define the API source.

2️⃣ Implement automatic pagination using dlt's built-in REST client.

3️⃣ Load the extracted data into DuckDB for querying.

In [5]:
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import PageNumberPaginator

# Defining the API function for NYC taxi data
@dlt.resource(name="rides")   # <--- The resource name 
def ny_taxi():
    client = RESTClient(
        base_url="https://us-central1-dlthub-analytics.cloudfunctions.net",
        paginator=PageNumberPaginator(
            base_page=1,
            total_path=None # <--- No total count of pages provided by API, pagination should stop when a page contains no result items
        )
    )

    for page in client.paginate("data_engineering_zoomcamp_api"):    # <--- API endpoint for retrieving taxi ride data
        yield page   # <--- yield data to manage memory


pipeline = dlt.pipeline(
    pipeline_name="ny_taxi_pipeline",
    destination="duckdb",
    dataset_name="ny_taxi_data"
)

**Load the data into DuckDB to test:**

In [6]:
load_info = pipeline.run(ny_taxi)
print(load_info)

2025-02-16 22:59:50,218|[WARNING]|57188|8282817088|dlt|configuration.py|_path_from_pipeline:178|Duckdb attached to pipeline ny_taxi_pipeline in path ny_taxi_pipeline.duckdb was could not be found but pipeline has already ran. This may be a result of (1) recreating or attaching pipeline  without or with changed explicit path to database that was used when creating the pipeline. (2) keeping the path to to database in secrets and changing the current working folder so  dlt cannot see them. (3) you deleting the database.


Pipeline ny_taxi_pipeline load step completed in 1.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset ny_taxi_data
The duckdb destination used duckdb:////Users/blessingangus/Downloads/data_engineering/personal_de_projects/DE_Zoomcamp/de-zoomcamp-2025/data_ingestion_with_dlt/ny_taxi_pipeline.duckdb location to store data
Load package 1739743190.390413 is LOADED and contains no failed jobs



**Start a connection to your database using native duckdb connection and look what tables were generated:**

In [7]:
import duckdb

# A database '<pipeline_name>.duckdb' was created in working directory so just connect to it

# Connect to the DuckDB database
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# Set search path to the dataset
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

# Describe the dataset
conn.sql("DESCRIBE").df()

,database,schema,name,column_names,column_types,temporary
0,ny_taxi_pipeline,ny_taxi_data,_dlt_loads,"[load_id, schema_name, status, inserted_at, sc...","[VARCHAR, VARCHAR, BIGINT, TIMESTAMP WITH TIME...",False
1,ny_taxi_pipeline,ny_taxi_data,_dlt_pipeline_state,"[version, engine_version, pipeline_name, state...","[BIGINT, BIGINT, VARCHAR, VARCHAR, TIMESTAMP W...",False
2,ny_taxi_pipeline,ny_taxi_data,_dlt_version,"[version, engine_version, inserted_at, schema_...","[BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, VAR...",False
3,ny_taxi_pipeline,ny_taxi_data,rides,"[end_lat, end_lon, fare_amt, passenger_count, ...","[DOUBLE, DOUBLE, DOUBLE, BIGINT, VARCHAR, DOUB...",False


## **Question 3: Explore the loaded data**

Inspect the table `ride`:

```py
df = pipeline.dataset(dataset_type="default").rides.df()
df
```

What is the total number of records extracted?

* 2500
* 5000
* 7500
* 10000

Answer: **10000**

In [8]:
df = pipeline.dataset(dataset_type="default").rides.df()
df

,end_lat,end_lon,fare_amt,passenger_count,payment_type,start_lat,start_lon,tip_amt,tolls_amt,total_amt,trip_distance,trip_dropoff_date_time,trip_pickup_date_time,surcharge,vendor_name,_dlt_load_id,_dlt_id,store_and_forward
0,40.742963,-73.980072,45.0,1,Credit,40.641525,-73.787442,9.0,4.15,58.15,17.52,2009-06-14 23:48:00+00:00,2009-06-14 23:23:00+00:00,0.0,VTS,1739743190.390413,Jbry49epI3rOtw,NaN
1,40.740187,-74.005698,6.5,1,Credit,40.722065,-74.009767,1.0,0.00,8.50,1.56,2009-06-18 17:43:00+00:00,2009-06-18 17:35:00+00:00,1.0,VTS,1739743190.390413,zADdk9Mvyn4Ciw,NaN
2,40.718043,-74.004745,12.5,5,Credit,40.761945,-73.983038,2.0,0.00,15.50,3.37,2009-06-10 18:27:00+00:00,2009-06-10 18:08:00+00:00,1.0,VTS,1739743190.390413,YCPw1sOwTOC5zg,NaN
3,40.739637,-73.985233,4.9,1,CASH,40.749802,-73.992247,0.0,0.00,5.40,1.11,2009-06-14 23:58:00+00:00,2009-06-14 23:54:00+00:00,0.5,VTS,1739743190.390413,OWcabqxwciGjuw,NaN
4,40.730032,-73.852693,25.7,1,CASH,40.776825,-73.949233,0.0,4.15,29.85,11.09,2009-06-13 13:23:00+00:00,2009-06-13 13:01:00+00:00,0.0,VTS,1739743190.390413,Ii1cM//bvSdO6g,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,40.783522,-73.970690,5.7,1,CASH,40.778560,-73.953660,0.0,0.00,5.70,1.16,2009-06-19 11:28:00+00:00,2009-06-19 11:22:00+00:00,0.0,VTS,1739743190.390413,tSJq3rGULNu0Cw,NaN
9996,40.777200,-73.964197,4.1,1,CASH,40.779800,-73.974297,0.0,0.00,4.10,0.89,2009-06-17 07:43:00+00:00,2009-06-17 07:41:00+00:00,0.0,VTS,1739743190.390413,7g3NCtLwH4yFNg,NaN
9997,40.780172,-73.957617,6.1,1,CASH,40.788388,-73.976758,0.0,0.00,6.10,1.30,2009-06-19 11:46:00+00:00,2009-06-19 11:39:00+00:00,0.0,VTS,1739743190.390413,6VCJudYQDRJwJg,NaN
9998,40.777342,-73.957242,5.7,1,CASH,40.773828,-73.956690,0.0,0.00,6.20,0.97,2009-06-17 04:19:00+00:00,2009-06-17 04:13:00+00:00,0.5,VTS,1739743190.390413,n35v3y/rMI3jYQ,NaN


## **Question 4: Trip Duration Analysis**

Run the SQL query below to:

* Calculate the average trip duration in minutes.

```py
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM rides;
            """
        )
    # Prints column values of the first row
    print(res)
```

What is the average trip duration?

* 12.3049
* 22.3049
* 32.3049
* 42.3049

In [9]:
with pipeline.sql_client() as client:
    res = client.execute_sql(
            """
            SELECT
            AVG(date_diff('minute', trip_pickup_date_time, trip_dropoff_date_time))
            FROM rides;
            """
        )
    # Prints column values of the first row
    print(res)

[(12.3049,)]
